In [27]:
import cv2
print(cv2.__version__)
import numpy as np
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import confusion_matrix
from sklearn.ensemble import RandomForestClassifier
import imutils
import sys
import numpy
from os import listdir
from os.path import isfile, join
from tqdm import tqdm
numpy.set_printoptions(threshold=sys.maxsize)

4.1.1


## Read images

In [2]:
img_full = cv2.imread('full_image.jpg')
img_bkg = cv2.imread('selected_data/background.jpg')
img_pallet = cv2.imread('selected_data/pallet.jpg')

## Define showing and features functions

In [3]:
def show(img):
 cv2.imshow('image',img)
 cv2.waitKey(0)
 cv2.destroyAllWindows()

In [4]:
def bgr_to_cmyk(bgr):
    RGB_SCALE = 255
    CMYK_SCALE = 100
    b = bgr[0]
    g = bgr[1]
    r = bgr[2]
    if (r, g, b) == (0, 0, 0):
        return 0, 0, 0, CMYK_SCALE

    c = 1 - r / RGB_SCALE
    m = 1 - g / RGB_SCALE
    y = 1 - b / RGB_SCALE

    min_cmy = min(c, m, y)
    c = (c - min_cmy) / (1 - min_cmy)
    m = (m - min_cmy) / (1 - min_cmy)
    y = (y - min_cmy) / (1 - min_cmy)
    k = min_cmy

    return [c, m, y, k]

def flatten(img):
    return np.reshape(img, (img.shape[0]*img.shape[1],img.shape[2]))

def features(bgr, channels, filter_white=True):
    hsv = cv2.cvtColor(bgr, cv2.COLOR_BGR2HSV)
    cmyk = np.apply_along_axis(bgr_to_cmyk, 2, bgr)
    lab = cv2.cvtColor(bgr, cv2.COLOR_BGR2LAB)
    
    hsv_flat = flatten(hsv)
    lab_flat = flatten(lab)
    cmyk_flat = flatten(cmyk)
    bgr_flat = flatten(bgr)
    
    f = np.concatenate((bgr_flat, hsv_flat, cmyk_flat, lab_flat),1)
    f = [x for x in f if not np.sum(x[:3] == [255,255,255]) == 3] #remove white pixels
    f = np.array(f)
    channels_map = {'B':0, 'G':1,'R':2,'H':3,'S':4,'V':5,'C':6,'M':7,'Y':8,'K':9,'L':10,'A':11,'b':12}
    channels = list(channels)
    channels = [channels_map[ch] for ch in channels]
    f = f[:,channels]
    return np.array(f)

## Select features (channels)

In [5]:
channels = "HSCMYb"

## Create train and test dataset

In [6]:
img_full_features = features(img_full, channels)
img_bkg_features = features(img_bkg, channels)
img_pallet_features = features(img_pallet, channels)

In [7]:
X = np.concatenate((img_bkg_features, img_pallet_features))
y = np.concatenate(([0]*len(img_bkg_features), [1]*len(img_pallet_features)))

In [8]:
sss = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=0)
sss.get_n_splits(X, y)
      
train_index, test_index = sss.split(X, y).__next__()
X_train, X_test = X[train_index], X[test_index]
y_train, y_test = y[train_index], y[test_index]

## Gaussian Naive Bayes - train and test

In [9]:
gnb = GaussianNB()
gnb.fit(X_train, y_train)
y_pred=gnb.predict(X_test)
confusion_matrix(y_test, y_pred)

array([[600053, 190177],
       [183785, 311439]])

## Random Forest - train and test (slower)

In [36]:
# clf = RandomForestClassifier(n_estimators=300, max_depth=10, random_state=0)
# clf.fit(X_train, y_train)
# y_pred=clf.predict(X_test)
# print(clf.feature_importances_)
# confusion_matrix(y_test, y_pred)

[0.12887468 0.13561565 0.13638794 0.20813514 0.23945653 0.15153006]


array([[682104, 108126],
       [188143, 307081]])

## Define pipeline functions

In [11]:
def detect(img, model):
    w,h,ch = img.shape
    f = features(img, channels, False)
    pred = model.predict(f)
    img_pred = np.reshape(pred, (w,h,1))
    return np.logical_not(img_pred)

In [12]:
def median_filter(img):
    kernel = np.ones((5,5),np.float32)/25
    img = cv2.filter2D(img,-1,kernel)
    return cv2.threshold(img,0.5,1.0,cv2.THRESH_BINARY)[1]

In [13]:
def rotate(img, angle):
    return  imutils.rotate(img, angle)

In [18]:
def find_pallet_by_height(img, mask_height):
    mask_height = mask_height
    mask_width = int(mask_height * 5.556)
    mask_size = mask_height * mask_width
    hole_height = int(mask_height * 0.694)
    hole_width = int(mask_height * 1.58)
    hole_size = hole_height * hole_width
    hole_1_x = int(mask_height * 0.694)
    hole_1_y = int(mask_height * 0.306)
    hole_2_x = int(mask_height * 3.281)
    hole_2_y = hole_1_y
    best = (None, 0, (0,0))
    inverse_img = np.logical_not(img)
    for index, x in np.ndenumerate(inverse_img):
        x,y = index
        frame_mask = inverse_img[y:y+mask_height, x:x+mask_width]
        hole_1_y_ = y+hole_1_y
        hole_2_y_ = y+hole_2_y
        hole_1_x_ = x+hole_1_x
        hole_2_x_ = x+hole_2_x
        hole_1_mask = inverse_img[hole_1_y_:hole_1_y_+hole_height, hole_1_x_:hole_1_x_+hole_width]
        hole_2_mask = inverse_img[hole_2_y_:hole_2_y_+hole_height, hole_2_x_:hole_2_x_+hole_width]
        frame_mask_perc = np.sum(frame_mask) / mask_size
        hole_1_mask_perc = np.sum(hole_1_mask) / hole_size
        hole_2_mask_perc = np.sum(hole_2_mask) / hole_size
        perc = frame_mask_perc - hole_1_mask_perc - hole_2_mask_perc
        if perc >= best[1]:
            best = (frame_mask, perc, (x,y))
    return best

def find_pallet(img, min_height, max_height, step):
    best = (None, 0, (0,0,0))
    for mask_height in range(min_height, max_height, step):
        (best_for_height, perc, (x,y)) = find_pallet_by_height(img, mask_height)
        if perc >= best[1]:
            print(mask_height)
            best = (best_for_height, perc, (x,y,mask_height))
    
    return (np.logical_not(best[0]), best[2])

def draw_pallet(img,x,y,mask_height):
    mask_width = int(mask_height * 5.556)
    color = (0,255,0)
    img_full[y:y+mask_height, x:x+1]=color
    img_full[y:y+mask_height, x+mask_width-1:x+mask_width]=color
    img_full[y:y+1, x:x+mask_width]=color
    img_full[y+mask_height-1:y+mask_height, x:x+mask_width]=color
    return img_full

In [15]:
def save(img, file, binary=True):
    res = cv2.imwrite(file, img * 255 if binary else img)
    print("saved" if res else "save error", file)

## Run pipeline and save results

In [37]:
img_full = cv2.imread('r_3_272.jpg') #r_1_26.jpg

img_classified = detect(img_full, clf).astype('float32')
save(img_classified, "img_classified.jpg")

img_filtered = median_filter(img_classified)
save(img_filtered, "img_filtered.jpg")

img_rotated = rotate(img_classified, -2) #fixme
save(img_rotated, "img_rotated.jpg")

img_pallet, (x,y,mask_height) = find_pallet(img_rotated, 15, 30, 2)
save(img_pallet, "img_pallet.jpg")

img_full_marked = draw_pallet(img_full,x,y,mask_height)
save(img_full_marked, "img_full_marked.jpg", False)

saved img_classified.jpg
saved img_filtered.jpg
saved img_rotated.jpg
15
17
23
saved img_pallet.jpg
saved img_full_marked.jpg


# Pipeline as function

In [34]:
def localise(filename):
    img_full = cv2.imread(filename)
    img_classified = detect(img_full, gnb).astype('float32')
    #img_filtered = median_filter(img_classified)
    img_rotated = rotate(img_filtered, -2)
    img_pallet, (x,y,mask_height) = find_pallet(img_rotated, 15, 100, 3)
    img_full_marked = draw_pallet(img_full,x,y,mask_height)
    save(img_full_marked, "localised/"+filename, False)

In [35]:
path = "/home/maciej/repos/pallet-recognition/data/jpeg"
filenames = [f for f in listdir(path) if isfile(join(path, f))]
for i in tqdm(range(len(filenames))):
    localise(path+"/"+filenames[i])


  0%|          | 0/595 [00:00<?, ?it/s]

15


KeyboardInterrupt: 

## Notes
Must be todo's:
- train on many different video frames
- rotate frame
  
Improvements
- try add new channel feature

In [ ]:
#Results for channels combinations (Naive Bayes)
#all
array([[53592,  1409],
       [ 3527,  1784]]) #5311
#HSCY
array([[52410,  2591],
       [ 1819,  3492]])
#HSCMY
array([[52307,  2694],
       [ 1742,  3569]])
#HSMY
array([[52845,  2156],
       [ 2697,  2614]])
#HCMY
array([[53081,  1920],
       [ 3070,  2241]])
#SCMY
array([[51814,  3187],
       [ 2338,  2973]])
#HSCMYb
array([[51265,  3736],
       [ 1250,  4061]])
#RGBVK psują